In [1]:
import numpy as np
import torch
import joblib
from stuff.found.mnist1d.data import get_dataset, get_dataset_args
from glio.data import DSClassification, DSToTarget
from glio.torch_tools import CUDA_IF_AVAILABLE

__all__ = ["get_mnist1d_classification", "get_mnist1d_autoenc"]
def _loader(x, device=CUDA_IF_AVAILABLE):
    return torch.as_tensor(x, dtype=torch.float32, device=device).unsqueeze(0)

def get_mnist1d_classification(path='D:/datasets/mnist1d_data.pkl', download=False, device=CUDA_IF_AVAILABLE):
    args = get_dataset_args()
    data = get_dataset(args, path, download=download)
    dstrain = DSClassification()
    dstest = DSClassification()
    x = data['x']
    y = data['y']
    xtest = data['x_test']
    ytest = data['y_test']
    for sx, sy in zip(x, y):
        dstrain.add_sample(data = sx, target = torch.nn.functional.one_hot(torch.from_numpy(np.array([sy])), 10)[0].to(device), loader = lambda x: _loader(x, device=device), target_encoder=None) # pylint:disable=E1102

    for sx, sy in zip(xtest, ytest):
        dstest.add_sample(data = sx, target = torch.nn.functional.one_hot(torch.from_numpy(np.array([sy])), 10)[0].to(device), loader = lambda x: _loader(x, device=device), target_encoder=None)# pylint:disable=E1102

    dstrain.preload(); dstest.preload()
    return dstrain, dstest

def get_mnist1d_autoenc(path='D:/datasets/mnist1d_data.pkl', download=False, device=CUDA_IF_AVAILABLE):
    args = get_dataset_args()
    data = get_dataset(args, path, download=download)
    dstrain = DSToTarget()
    dstest = DSToTarget()
    dstrain.add_samples(data = data["x"], loader = lambda x: _loader(x, device=device))

    dstest.add_samples(data = data['x_test'], loader = lambda x: _loader(x, device=device))

    dstrain.preload(); dstest.preload()
    return dstrain, dstest

dstrain, dstest = get_mnist1d_classification()

Successfully loaded data from D:/datasets/mnist1d_data.pkl


In [2]:
dstrain[656]

(tensor([[-0.9939, -0.7080, -0.5977, -0.3484, -0.3525, -0.6252, -0.5411, -0.6469,
          -0.7733, -0.3849,  0.0781, -0.4516,  1.7370,  2.6350,  3.0601,  3.0837,
           2.4960,  1.5260,  0.0333,  0.5425,  0.8209,  0.8436,  1.0386,  1.4010,
           1.5657,  1.3732,  0.8543,  0.7989,  1.0642,  1.2924,  1.4934,  1.5826,
           1.2843,  0.9917,  1.0409,  1.2263,  1.7783,  1.7342,  1.9597,  2.4114]],
        device='cuda:0'),
 tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'))